In [1]:
import networkx as nx
import numpy as np

In [2]:
class GraphStructure(object):
    
    def __init__(self, nodes, edges):
        self.nodes = sorted(nodes)
        self.edges = sorted(edges)
       
    @classmethod
    def from_networkx(cls, graph):
        # cls == Graph
        # graph is a networkx object
        nodes = graph.nodes()
        edges = graph.edges()
        obj = cls(nodes, edges)
        return obj
    
    def to_networkx(self):
        graph = nx.Graph()
        graph.add_nodes(self.nodes)
        graph.add_edges(self.edges)
        return graph        
        
    def __eq__(self, other):
        if self.nodes != other.nodes:
            return False
        if self.edges != other.edges:
            return False
        return True
    
    def children(self, node):
        return [e for e in self.edges if e[0] == node]

    def parents(self, node):
        return [e for e in self.edges if e[1] == node]

In [3]:
class GraphParams(object):
    
    def __init__(self, n, lambda0=1.0, p=0.8):
        self.n = n             # number of edges
        self.lambda0 = lambda0 # scale-free parameter
        self.p = p             # probability of sending a message
        self.psi = None        # psi edge parameters
        self.r = None          # r edge parameters
        self.mu = None         # psi / r
         
    def sample(self):
        self.psi = np.random.gamma(shape=1.0, scale=self.lambda0, size=self.n)
        self.r = np.random.gamma(shape=1.0, scale=self.lambda0, size=self.n)
        self.mu = self.psi / self.r
        return self.to_dict()
    
    def to_dict(self):
        return {
            "n": self.n,
            "psi": self.psi,
            "r": self.r,
            "lambda0": self.lambda0,
            "p": self.p,
            "mu": self.mu
        }
    
    @classmethod
    def from_dict(cls, d):
        obj = cls(d['n'], lambda0=d['lambda0'], p=d['p'])
        obj.psi = d['psi']
        obj.r = d['r']
        return obj
    
    def update(self, d):
        for param, val in d.items():
            if param == 'mu':
                continue
            if hasattr(self, param):
                setattr(self, param, val)
            else:
                raise AttributeError("no such attribute '{}'".format(param))

In [4]:
class InnerGraphSimulation(object):
    
    def __init__(self, structure, params):
        self.structure = structure
        self.params = params
        self._all_events = None
        self._first_events = None
        
    def sample_edge(self, edge, time):
        index = self.structure.edges.index(edge)
        
        # does it occur?
        occurs = np.random.rand() < self.params.p
        if not occurs:
            return []
        
        # how many events?
        num_events = np.random.poisson(lam=self.params.mu[index])
        if num_events == 0:
            return []
        
        # when do those events occur?
        event_times = time + np.random.exponential(scale=self.params.r[index], size=num_events)
        event_times.sort()
        return [(t, edge[1]) for t in event_times]
        
    def _sample(self, first_only=True, max_time=4.0):
        pending = [(0, self.structure.nodes[0])]
        self._all_events = []
        self._first_events = None
        if first_only:
            processed_nodes = []

        while len(pending) > 0:
            time, node = pending.pop(0)
            if time >= max_time:
                break

            self._all_events.append((time, node))
            if first_only:
                if node in processed_nodes:
                    continue
                processed_nodes.append(node)

            children = self.structure.children(node)
            for edge in children:
                child_events = self.sample_edge(edge, time)
                if len(child_events) == 0:
                    continue
                pending.extend(child_events)
            pending.sort()
            
        self._all_events.sort()
        self._compute_first_events()
        return self._first_events
    
    def sample(self, k=1, first_only=True, max_time=4.0):
        first_events = np.empty((k, len(self.structure.nodes)))
        for i in range(k):
            first_events[i] = self._sample(first_only=first_only, max_time=max_time)
        return first_events
    
    def _compute_first_events(self):
        first_events = {node: np.inf for node in self.structure.nodes}
        for time, node in self._all_events:
            if first_events[node] < np.inf:
                continue
            first_events[node] = time
        self._first_events = np.array([first_events[node] for node in self.structure.nodes])

In [5]:
# create a fully connected graph structure
nodes = ["A", "B", "C", "D"]
edges = []
for node1 in nodes:
    for node2 in nodes:
        if node1 == node2:
            continue
        edges.append((node1, node2))
        
gs = GraphStructure(nodes, edges)

In [6]:
# graph parameters
gp = GraphParams(len(edges))
gp.sample()

{'lambda0': 1.0,
 'mu': array([  2.29344191e+00,   1.78416592e+00,   3.72334822e-01,
          2.24343280e-02,   1.37785879e+00,   6.09284345e+00,
          4.57713701e+00,   1.47412899e-01,   1.93547415e-01,
          1.08081522e+02,   5.39666128e-02,   1.13605122e+00]),
 'n': 12,
 'p': 0.8,
 'psi': array([ 2.26187893,  0.22476094,  0.61717296,  0.0956303 ,  0.66182561,
         1.40993516,  0.21701461,  0.30007726,  0.46604834,  2.05857858,
         0.10537935,  2.0718588 ]),
 'r': array([ 0.98623773,  0.12597535,  1.65757519,  4.26267733,  0.48032905,
         0.2314084 ,  0.04741274,  2.03562415,  2.40792851,  0.01904654,
         1.95267672,  1.82373713])}

In [7]:
# inner graph simulation
sim = InnerGraphSimulation(gs, gp)
sim.sample(1000)

array([[ 0.        ,  0.41158525,  0.06832821,  0.41353685],
       [ 0.        ,  0.20532481,  0.01110261,  0.25971305],
       [ 0.        ,  0.4155097 ,  0.62615129,  0.17583114],
       ..., 
       [ 0.        ,         inf,  0.10292434,         inf],
       [ 0.        ,         inf,  0.6786789 ,         inf],
       [ 0.        ,         inf,  0.07603071,  0.42576558]])

In [8]:
sim._all_events

[(0, 'A'),
 (0.076030710976054863, 'C'),
 (0.092917495627482777, 'A'),
 (0.14298321402152042, 'A'),
 (0.42576557806129844, 'D'),
 (0.42623086640203062, 'A'),
 (0.42637169418479687, 'A'),
 (0.42673504999797218, 'A'),
 (0.42701250458899476, 'A'),
 (0.42711696622241585, 'A'),
 (0.42727368255344039, 'A'),
 (0.42768900089396034, 'A'),
 (0.42781915446310509, 'A'),
 (0.42782167558070938, 'A'),
 (0.42810358114416341, 'A'),
 (0.428221875136789, 'A'),
 (0.42848186842007835, 'A'),
 (0.42870404524377215, 'A'),
 (0.4289233680198441, 'A'),
 (0.43000859460216567, 'A'),
 (0.4302352309972155, 'A'),
 (0.43045424956369233, 'A'),
 (0.43048510924313466, 'A'),
 (0.43192758290776251, 'A'),
 (0.432412339849995, 'A'),
 (0.43280436478818723, 'A'),
 (0.43384784946238752, 'A'),
 (0.43473899125738824, 'A'),
 (0.43509606941779061, 'A'),
 (0.43513932259679255, 'A'),
 (0.43517810660627065, 'A'),
 (0.43534116354054891, 'A'),
 (0.43587313827652563, 'A'),
 (0.43588701856543532, 'A'),
 (0.43694359001014199, 'A'),
 (0.437

In [ ]:
class GraphSimulation(object):
    
    def __init__(self, events):
        self.inner = 
        
    def likelihood(self, data):
        # compute likelihood given sample from 